# CE-40719: Deep Learning
## HW3 - CNN / CNN Case Studies / CNN Applications
(18 points)

#### Name:
#### Student No.:

In this exercise we are going to implement a simple residual netwrok architecture to classify images from Cifar10 dataset. Here we give you a suggestion for architecture but you are allowed to  make changes and experiment to get better results. Explain your ideas or reference the papers that you take the ideas from. You are allowed to use out-of-the-box pytorch modules in `torch.nn`.

## Architecture

_All convolution layers have `3 * 3` kernel,  `padding=1`, batch normalization and relu activation_

__ResidualBlock:__ _in_channel, out_channel, stride_

- Conv(in_channel, out_channel, stride)
- Conv(out_channel, out_channel, stride=1)

`*` This block has a residual connection. To match dimmensions of output and residual use `1 * 1` convolution and stride. 

__ResidualLayer:__ _in_channel, out_channel, stride_

- ResidualBlock(in_channel, out_channel, stride)
- ResidualBlock(out_channel, out_channel, stride=1)

__ResidualNetwork__:
- Conv(3, 64, stride=1)
- ResidualLayer(64, 64, stride=1)
- ResidualLayer(64, 128, stride=2)
- ResidualLayer(128, 256, stride=2)
- ResidualLayer(256, 512, stride=2)
- AveragePool(4, 4)
- Linear(512, 10)


In [0]:
pass